In [1]:
!pip install ortools openpyxl pandas

In [38]:
# -*- coding: utf-8 -*-
"""
Capacitated VRP (CVRP) - OR-Tools versiyonu
- Mesafe matrisi Excel'den okunuyor: /content/kcekmece_distance.xlsx
- 0 mesafeli (i != j) girişler 'yol yok' olarak kabul edilip BIG_M cezası veriliyor.
- MANDATORY_NODES: talebi 1 olan düğümler (diğer tüm düğümler talep=0).
- VEHICLE_CAPACITIES: araç kapasite vektörü.
"""

import pandas as pd
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# ==============================
# KULLANICI PARAMETRELERİ
# ==============================

EXCEL_PATH = "/content/kcekmece_distance.xlsx"
SHEET_NAME = "Sheet1"

# Depo index'i (Excel'deki node indekslerine göre)
DEPOT = 77  # örnek

# 1 birim talep olan düğümler (diğer tüm node'lar 0 talep)
# -> Bunları kendi senaryona göre değiştir!
MANDATORY_NODES = [464, 573, 607, 699]
# Araç kapasiteleri (her eleman o aracın max toplam talebi)
VEHICLE_CAPACITIES = [1, 1, 1, 1] # toplam = len(MANDATORY_NODES) olmasını istiyorsan buna göre ayarla
NUM_VEHICLES = len(VEHICLE_CAPACITIES)


# ==============================
# MESAFE MATRİSİ YARDIMCI FONKSİYONLAR
# ==============================

def load_distance_matrix_xlsx(path, sheet_name="Sheet1"):
    """Excel'den kare mesafe matrisi oku, gereksiz boş satır/sütunları temizle."""
    df = pd.read_excel(path, sheet_name=sheet_name, header=None)

    # Baştaki tamamen boş satırları temizle
    while df.shape[0] > 0 and df.iloc[0].isna().all():
        df = df.iloc[1:, :].reset_index(drop=True)

    # Baştaki tamamen boş sütunları temizle
    while df.shape[1] > 0 and df.iloc[:, 0].isna().all():
        df = df.iloc[:, 1:].reset_index(drop=True)

    df = df.apply(pd.to_numeric, errors='coerce').fillna(0.0)
    assert df.shape[0] == df.shape[1], f"Mesafe matrisi kare olmalı, şu an: {df.shape}"
    arr = df.values.astype(np.float64)

    # Diagonal 0
    n = arr.shape[0]
    for i in range(n):
        arr[i, i] = 0.0

    return arr


def preprocess_distance_matrix(dm_raw, multiplier=1000.0):
    """
    Orijinal dm_raw'dan OR-Tools'a uygun distance_matrix üretir:
    - i == j  -> 0
    - i != j ve dm_raw[i,j] > 0 -> int(round(dm_raw[i,j]))
    - i != j ve dm_raw[i,j] <= 0 -> BIG_M (yol yok / bilinmeyen)
    BIG_M = max_pozitif_mesafe * multiplier
    """
    arr = dm_raw.copy().astype(np.float64)
    n = arr.shape[0]

    positive = arr[arr > 0]
    if positive.size == 0:
        raise ValueError("Mesafe matrisinde pozitif değer yok, BIG_M seçilemedi.")

    # max_dist = float(positive.max())
    # big_m = int(max_dist * multiplier)
    big_m = 2500

    dm_int = np.zeros((n, n), dtype=np.int64)
    for i in range(n):
        for j in range(n):
            if i == j:
                dm_int[i, j] = 0
            else:
                val = arr[i, j]
                if val <= 0.0:
                    # Yol yok / 0 girilmiş -> devasa ceza
                    dm_int[i, j] = big_m
                else:
                    dm_int[i, j] = int(round(val))

    return dm_int, big_m


# ==============================
# DATA MODEL
# ==============================

def create_data_model():
    """Problemin verisini hazırlar (distance, demand, kapasite, depot, vs.)."""
    # 1) Excel'den ham mesafe matrisi
    print("[INFO] Excel'den mesafe matrisi okunuyor...")
    dm_raw = load_distance_matrix_xlsx(EXCEL_PATH, sheet_name=SHEET_NAME)
    n = dm_raw.shape[0]
    print(f"[INFO] Matris boyutu: {n} x {n}")

    # 2) BIG_M'li integer mesafe matrisi (0 -> BIG_M)
    distance_matrix_int, big_m = preprocess_distance_matrix(dm_raw, multiplier=1000.0)
    print(f"[INFO] BIG_M (yol yok cezası) = {big_m}")

    # 3) MANDATORY_NODES kontrol
    for idx in MANDATORY_NODES:
        if idx < 0 or idx >= n:
            raise ValueError(f"MANDATORY_NODES içindeki {idx} index'i matris boyutu {n} ile uyumsuz!")

    if DEPOT < 0 or DEPOT >= n:
        raise ValueError(f"DEPOT index'i ({DEPOT}) matris boyutu {n} ile uyumsuz!")

    # 4) Talep vektörü:
    #    - Default: tüm node'lar 0
    #    - MANDATORY_NODES: 1 birim talep
    demands = [0] * n
    for idx in MANDATORY_NODES:
        if idx == DEPOT:
            # Depo talep node olmasın, ama istersen burayı 0 bırakmak mantıklı
            print(f"[UYARI] MANDATORY_NODES içinde depo ({DEPOT}) var, talep atlanıyor.")
            continue
        demands[idx] = 1

    total_demand = sum(demands)
    total_capacity = sum(int(c) for c in VEHICLE_CAPACITIES)

    print(f"[INFO] Toplam talep            : {total_demand}")
    print(f"[INFO] Toplam araç kapasitesi  : {total_capacity}")
    print(f"[INFO] Araç kapasiteleri       : {VEHICLE_CAPACITIES}")
    print(f"[INFO] Depo (index)            : {DEPOT}")

    if total_capacity < total_demand:
        raise ValueError(
            f"Toplam kapasite ({total_capacity}), toplam talebin ({total_demand}) altında! "
            f"Araç kapasitelerini arttır."
        )

    data = {}
    data["distance_matrix"] = distance_matrix_int.tolist()
    data["dm_raw"] = dm_raw  # istersen sonradan orijinal mesafeyi görmek için
    data["big_m"] = big_m
    data["demands"] = demands
    data["vehicle_capacities"] = [int(c) for c in VEHICLE_CAPACITIES]
    data["num_vehicles"] = NUM_VEHICLES
    data["depot"] = DEPOT
    return data


# ==============================
# ÇÖZÜM YAZDIRMA
# ==============================

def print_solution(data, manager, routing, solution):
    """Çözümü konsola yazdırır."""
    print(f"\nObjective (toplam maliyet): {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0

    distance_matrix = data["distance_matrix"]
    dm_raw = data["dm_raw"]
    big_m = data["big_m"]

    for vehicle_id in range(data["num_vehicles"]):
        # Kullanılmayan aracı atla (OR-Tools gereksiz aracı boş bırakabilir)
        if not routing.IsVehicleUsed(solution, vehicle_id):
            print(f"\n[INFO] Vehicle {vehicle_id} kullanılmadı.")
            continue

        index = routing.Start(vehicle_id)
        plan_output = f"\nRoute for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand_here = data["demands"][node_index]
            route_load += demand_here
            plan_output += f" {node_index} (demand={demand_here}, load={route_load}) -> "

            previous_index = index
            index = solution.Value(routing.NextVar(index))

            # Arc cost (cezalı mesafe)
            arc_cost = routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            route_distance += arc_cost

        # Son node (depot)
        end_node = manager.IndexToNode(index)
        plan_output += f" {end_node} (demand=0, load={route_load})\n"
        plan_output += f"Distance of the route (model cost): {route_distance}\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)

        total_distance += route_distance
        total_load += route_load

    print(f"\nTotal distance of all routes (model cost): {total_distance}")
    print(f"Total load of all routes: {total_load}")

    # İsteğe bağlı: BIG_M kullanılan kenarları uyarı olarak göster
    print("\n[CHECK] BIG_M kullanılan (yol yok) edge var mı?")
    used_big_m_edges = []
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            from_index = index
            to_index = solution.Value(routing.NextVar(index))
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            if from_node != to_node:
                model_dist = distance_matrix[from_node][to_node]
                if model_dist >= big_m:
                    used_big_m_edges.append((vehicle_id, from_node, to_node))
            index = to_index

    if used_big_m_edges:
        print("\n[UYARI] Aşağıdaki kenarlarda BIG_M kullanıldı (orijinal dm_raw'da mesafe <= 0):")
        for (vid, u, v) in used_big_m_edges:
            print(f"  Vehicle {vid}: {u} -> {v} | dm_raw[{u},{v}] = {dm_raw[u, v]}")
    else:
        print("[INFO] Hiçbir araç rotasında BIG_M (yol yok) kenarı kullanılmadı.")


# ==============================
# MAIN: CVRP ÇÖZ
# ==============================

def main():
    # Veriyi hazırla
    data = create_data_model()

    # Routing index manager
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Routing model
    routing = pywrapcp.RoutingModel(manager)

    # Mesafe callback'i
    def distance_callback(from_index, to_index):
        """İki node arasındaki mesafeyi döner (BIG_M'li integer)."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Tüm araçlar için arc maliyeti
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Talep callback'i
    def demand_callback(from_index):
        """Node'un talebini döner."""
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    # Kapasite kısıtı
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data["vehicle_capacities"],  # kapasite vektörü
        True,  # start cumul to zero
        "Capacity",
    )

    # İlk çözüm + local search stratejileri
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    # Süre limiti (saniye)
    search_parameters.time_limit.FromSeconds(180)

    # Çöz
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("Çözüm bulunamadı! (Belki kapasite çok düşük, ya da BIG_M çok agresif)")

# Çalıştır
if __name__ == "__main__":
    main()


[INFO] Excel'den mesafe matrisi okunuyor...
[INFO] Matris boyutu: 822 x 822
[INFO] BIG_M (yol yok cezası) = 2500
[INFO] Toplam talep            : 4
[INFO] Toplam araç kapasitesi  : 4
[INFO] Araç kapasiteleri       : [1, 1, 1, 1]
[INFO] Depo (index)            : 77

Objective (toplam maliyet): 230223

Route for vehicle 0:
 77 (demand=0, load=0) ->  453 (demand=0, load=0) ->  451 (demand=0, load=0) ->  452 (demand=0, load=0) ->  697 (demand=0, load=0) ->  698 (demand=0, load=0) ->  109 (demand=0, load=0) ->  622 (demand=0, load=0) ->  621 (demand=0, load=0) ->  620 (demand=0, load=0) ->  619 (demand=0, load=0) ->  497 (demand=0, load=0) ->  494 (demand=0, load=0) ->  772 (demand=0, load=0) ->  767 (demand=0, load=0) ->  771 (demand=0, load=0) ->  459 (demand=0, load=0) ->  454 (demand=0, load=0) ->  455 (demand=0, load=0) ->  457 (demand=0, load=0) ->  458 (demand=0, load=0) ->  445 (demand=0, load=0) ->  450 (demand=0, load=0) ->  442 (demand=0, load=0) ->  448 (demand=0, load=0) ->  44